In [21]:
from CoRe_Dataloader_ECSG import load_pth_file,load_raw_from_pth_file
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset
import math
import torchinfo
import time
import numpy as np
import wandb
import datetime
from collections import OrderedDict
from sklearn.model_selection import train_test_split

In [22]:
raw_sgram_ds, raw_param_ds = load_raw_from_pth_file()
def get_new_test_train(p = 0.3):
    assert p < 1
    xtrain,xtest,ytrain,ytest = train_test_split(raw_sgram_ds.cpu().numpy(),raw_param_ds.cpu().numpy(),test_size = p)
    train_dataset = TensorDataset(torch.tensor(xtrain),torch.tensor(ytrain))
    if p == 0:
        test_dataset = TensorDataset(torch.tensor(xtrain),torch.tensor(ytrain))
    else:
        test_dataset = TensorDataset(torch.tensor(xtest), torch.tensor(ytest))
    return DataLoader(train_dataset,batch_size = 8, shuffle = True), DataLoader(test_dataset,batch_size = 64,shuffle = True)


In [23]:
import torchmetrics as metrics
acc = metrics.Accuracy(task="multiclass",num_classes=19).to("cuda:0")
auroc = metrics.AUROC(task = "multiclass",num_classes=19).to("cuda:0")


In [24]:
def new_accuracy(model:torch.nn.Module,dl:DataLoader):
    model.eval()
    raw_output = []
    parameters = []
    with torch.no_grad():
        for batch,(sg,params) in enumerate(dl):
            sg = sg.to("cuda:0").to(torch.float)
            sgsh = sg.shape
            sg = sg.view(sgsh[0], 1, sgsh[1], sgsh[2])

            params = params[:,0].to("cuda:0").to(torch.long)
            
            raw_output.append(model(sg).detach())
            parameters.append(params)
            
    model.train()
    output = torch.vstack(raw_output)
    parameters = torch.hstack(parameters)
    accuracy = acc(output,parameters)
    auc = auroc(output,parameters)
    return accuracy,auc


In [25]:
# import vit
# import vit_pytorch
from vit_pytorch import vit_for_small_dataset as vit_sd
from vit_pytorch import vit as simple_vit

In [26]:
def init_model():
    # return simple_vit.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=int(1024/2),
    #                depth=2,
    #                heads=8,
    #                mlp_dim=int(2048/2),
    #                channels=1).to("cuda:0")
    return vit_sd.ViT(image_size=400,
                   patch_size=20,
                   num_classes=19,
                   dim=1024,
                   depth=3,
                   heads=16,
                   mlp_dim=int(2048/2),
                   dropout = 0.1,
                   emb_dropout = 0.1,
                   channels=1).to("cuda:0")


In [27]:
model = init_model()


img = torch.randn(1,1, 400,400).to("cuda:0")

preds = model(img)  # (1, 1000)
print(preds)
torchinfo.summary(model, input_size=(1,1, 400, 40))

tensor([[ 0.1761, -0.1176, -0.0582, -0.0227, -0.6981, -0.0344,  0.2345, -0.5933,
          1.1695, -0.4646,  0.0839,  0.9832, -0.1001, -0.7190,  0.0979,  0.1236,
         -0.8274, -0.3403, -0.2315]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


C:\Users\aashr\AppData\Roaming\Python\Python310\site-packages\torch\storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)


Layer (type:depth-idx)                             Output Shape              Param #
ViT                                                [1, 19]                   411,648
├─SPT: 1-1                                         [1, 40, 1024]             --
│    └─Sequential: 2-1                             [1, 40, 1024]             --
│    │    └─Rearrange: 3-1                         [1, 40, 2000]             --
│    │    └─LayerNorm: 3-2                         [1, 40, 2000]             4,000
│    │    └─Linear: 3-3                            [1, 40, 1024]             2,049,024
├─Dropout: 1-2                                     [1, 41, 1024]             --
├─Transformer: 1-3                                 [1, 41, 1024]             --
│    └─ModuleList: 2-2                             --                        --
│    │    └─ModuleList: 3-4                        --                        6,298,625
│    │    └─ModuleList: 3-5                        --                        6,298,625
│    │

In [28]:
startlr = 3e-5
optimizer = optim.Adam(params=model.parameters(), lr=startlr)
optimizer1 = optim.NAdam(params=model.parameters(), lr=startlr)
step_scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[250], gamma=0.9)
# at the end of 600 epochs, the learning rate is 0.000,002,62
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=1, gamma=0.988)
scheduler_pl = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', factor=0.7, patience=35, verbose=True)
lossfn = nn.CrossEntropyLoss()


In [29]:
def train_eval_model(config,train_dl,test_dl,adam = True,nadam = False):
    tot_acc,auc = new_accuracy(model=model,dl = test_dl)
    max_acc = -1
    max_auc = -1
    for epoch in range(1,config.epochs+1):
        btime = time.time()
        ldl = len(train_dl)
        for batch,(sg,params) in enumerate(train_dl):
            stime = time.time()
            sg = sg.to("cuda:0").to(torch.float)
            sgsh = sg.shape
            sg = sg.view(sgsh[0],1,sgsh[1],sgsh[2])
            params = params[:,0].to("cuda:0").to(torch.long)
            optimizer.zero_grad()
            outputs = model(sg)
            loss = lossfn(outputs,params)
            loss.backward()
            optimizer.step() if adam else None
            optimizer1.step() if nadam else None
            wandb.log({"loss":loss.item(),"batch_accuracy":acc(outputs,params),"lr":scheduler.get_last_lr()[0],"epoch":epoch})
            print(f"{epoch:5}/{config.epochs:5} // {batch:5}/{ldl:5} | Loss: {loss.item():2.4},batch_accuracy:{acc(outputs,params):3.4}, last_total_accuracy: {tot_acc}, Maximum Accuracy {max_acc} last AUROC {auc} Max AUC {max_auc} lr:{scheduler.get_last_lr()[0]:1.5},Time per Batch: {time.time()-stime:1.2} seconds     ",end = "\r",flush=True)
            torch.cuda.empty_cache()
        tot_acc, auc = new_accuracy(model=model, dl=test_dl)
        scheduler.step()
        step_scheduler.step()
        scheduler_pl.step(tot_acc)
        if(tot_acc > max_acc):
            max_acc = tot_acc
            config.best_model = model.state_dict()
            try:
                torch.save(config.best_model, f"./saved_models/ViT/best_model_state_dict_ViT_for{config.run_name}_stime_{config.start_time.replace(':', '-')}__acc_{max_acc}__auc_{auc}.pt")
            except:  pass    
            print("\nSAVING MODEL")
        max_auc = max(max_auc,auc)
        print(f"\nEpoch {epoch+1}/{config.epochs} finished. Total accuracy: {tot_acc:3.5} AUROC: {auc} Time per Epoch: {time.time()-btime:1.5}")

        wandb.log({"epoch":epoch,"accuracy":tot_acc,"max_accuracy":max_acc,"lr":scheduler.get_last_lr()[0],"auroc":auc})
    return max_acc,max_auc

In [30]:
results = []
trials = 5

In [31]:
for i in range(trials):
    wandb.init(project="simple_vision_transformer_forsmalldatasets_validation_test")
    config = wandb.config
    config.run_name = wandb.run._run_id
    config = wandb.config
    config.epochs = 500
    config.inx = 400
    config.iny = 400
    config.lr = startlr     
    config.trial = i+1
    config.total_trials = trials
    config.best_model = OrderedDict()
    config.start_time = datetime.datetime.now().isoformat()
    config.savename = f"best_model_state_dict_at_for{config.run_name}_stime_{config.start_time.replace(':', '-')}__acc_max_acc__auc_auc.pt"
    train_dl, test_dl = get_new_test_train(.05)
    results.append(train_eval_model(wandb.config,train_dl,test_dl,nadam=True))
    model = init_model()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aashraychegu (alabs). Use `wandb login --relogin` to force relogin


C:\Users\aashr\AppData\Roaming\Python\Python310\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


    1/  500 //   280/  281 | Loss: 1.497,batch_accuracy:0.8333, last_total_accuracy: 0.008403361774981022, Maximum Accuracy -1 last AUROC 0.35685214400291443 Max AUC -1 lr:3e-05,Time per Batch: 0.11 seconds     
SAVING MODEL

Epoch 2/500 finished. Total accuracy: 0.43697 AUROC: 0.44416216015815735 Time per Epoch: 42.132
    2/  500 //   280/  281 | Loss: 1.103,batch_accuracy:0.5, last_total_accuracy: 0.43697479367256165, Maximum Accuracy 0.43697479367256165 last AUROC 0.44416216015815735 Max AUC 0.44416216015815735 lr:2.964e-05,Time per Batch: 0.11 seconds        
SAVING MODEL

Epoch 3/500 finished. Total accuracy: 0.47899 AUROC: 0.45062416791915894 Time per Epoch: 41.462
    3/  500 //   280/  281 | Loss: 2.442,batch_accuracy:0.3333, last_total_accuracy: 0.4789915978908539, Maximum Accuracy 0.4789915978908539 last AUROC 0.45062416791915894 Max AUC 0.45062416791915894 lr:2.9284e-05,Time per Batch: 0.11 seconds     
SAVING MODEL

Epoch 4/500 finished. Total accuracy: 0.61345 AUROC: 0.57

In [ ]:
a = []
for i in results:
    a.append([i[0].cpu().item(),i[1].cpu().item()])

In [ ]:
results = np.array(a)
average_accuracy = np.average(results[:,0])
average_auroc = np.average(results[:,1])
print(average_accuracy,average_auroc)

In [ ]:
torch.save(config.best_model,
           f"./saved_models/ViT/best_model_state_dict_at_for{config.run_name}_stime_{config.start_time.replace(':', '-')}_BEST_MODEL.pt")
